# A Real-Time Dengue Map for Singapore
- README.md

## Setup and imports 
- libraries, path setup
- dependencies.txt

### Key Dependencies
```python
# Core libraries for GIS and data processing
import pandas as pd
import geopandas as gpd
import httpx  # For API requests
import json
from datetime import datetime, timedelta
from pathlib import Path

# For spatial analysis and interpolation
import rasterio
import pyproj
from shapely.geometry import Point, Polygon
from sklearn.preprocessing import MinMaxScaler

# For web dashboard
from fastapi import FastAPI
import uvicorn
```

## Data acquisition 
> Acquire Live Data from APIs
- api_config(api sources)
- fetch_dengue.py, fetch_weather.py
- subzoneboundaries, populationsubzone
- fetch_ndvi.py

## Processing & Cleaning
> Cleaning data and applying temporal lag

**Key Processing Steps:**
1. **Temporal Lag Application** - Apply 2-month lag to weather data for dengue modeling
2. **Spatial Projection** - Project all data to SVY21 coordinate system (Singapore's national grid)
3. **Population Density** - Calculate population density per subzone
4. **Surface Interpolation** - Create continuous surfaces using Kriging for weather data
5. **NDVI Processing** - Calculate vegetation density from satellite imagery

**Scripts:**
- `apply_temporal_lag.py` - Process weather data with 2-month lag
- `calculate_population_density.py` - Compute population densities by subzone
- `interpolate_surfaces.py` - Spatial interpolation of weather surfaces
- `create_ndvi.py` - Vegetation density calculation

## Analysis (Prepare Data)
> Aggregation and risk calculation

**Analytical Hierarchy Process (AHP) Variables:**
1. **Dengue Cases** - Current cluster data (Red/Yellow/Green zones)
2. **Lagged Rainfall** - 2-month lagged precipitation data  
3. **Lagged Air Temperature** - 2-month lagged temperature (focus on minimum values)
4. **Population Density** - Residents per subzone area
5. **Vegetation Density (NDVI)** - Normalized Difference Vegetation Index

**Data Aggregation:**
- All variables aggregated to subzone level
- Spatial interpolation for weather surfaces
- Population density calculation from census data
- NDVI derived from satellite imagery
- Risk scores normalized and weighted using AHP methodology

## Weighted Risk Score Calculation

**Analytical Hierarchy Process (AHP) Methodology:**
- Determine variable weights through pairwise comparison
- Validate weights using space-time scan statistics on historical data
- Calculate weighted risk scores for each subzone

**Risk Factors:**
1. **Current Dengue Activity** (highest weight)
2. **Lagged Weather Conditions** (temperature + rainfall)
3. **Population Density** (exposure factor)
4. **Vegetation Density** (breeding habitat indicator)

**Output:** Continuous risk surface for real-time dengue mapping

## Visualization & Deployment
> Creating maps and statistical visualizations

**Web Dashboard Components:**
- **Interactive Map**: Leaflet.js with risk surface overlays
- **Real-time Updates**: 5-minute refresh cycle for weather, daily for dengue
- **Risk Legend**: Color-coded risk levels (Red/Yellow/Green + gradient)
- **Data Panels**: Current statistics and trend indicators

**Technical Stack:**
- **Backend**: FastAPI (Python) for data processing and API endpoints
- **Frontend**: HTML5/JavaScript with Leaflet mapping library
- **Data Flow**: RESTful APIs serving GeoJSON risk surfaces
- **Deployment**: Local development server with auto-reload

**Configuration Files:**
- `app_config.json` - Dashboard settings and refresh intervals
- `api_config.json` - Data source endpoints and credentials

## Reference

## Credits